# Criar forms no banco de dados 

Deve-se executar as duas primeiras células para carregar funções utilizadas para rodar o script e verificar se está em Produção

In [9]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from tqdm.notebook import tqdm

In [10]:
!cat .env

STAGE=PRODUCTION


Criando diretório com nome do arquivo

In [11]:
!mkdir "Modera v2"

Movendo arquivo .zip para o diretório

In [12]:
!mv "Modera v2.zip" "Modera v2"/

Descompactando arquivo .zip

In [13]:
!cd "Modera v2"/ && unzip "Modera v2.zip"

Archive:  Modera v2.zip
  inflating: 82.txt                  
  inflating: 83.txt                  
  inflating: 84.txt                  
  inflating: 85.txt                  
  inflating: 86.txt                  
  inflating: 87.txt                  
  inflating: 88.txt                  
  inflating: Modera v2.xlsx          
  inflating: 1.txt                   
  inflating: 2.txt                   
  inflating: 3.txt                   
  inflating: 4.txt                   
  inflating: 5.txt                   
  inflating: 6.txt                   
  inflating: 7.txt                   
  inflating: 8.txt                   
  inflating: 9.txt                   
  inflating: 10.txt                  
  inflating: 11.txt                  
  inflating: 12.txt                  
  inflating: 13.txt                  
  inflating: 14.txt                  
  inflating: 15.txt                  
  inflating: 16.txt                  
  inflating: 17.txt                  
  inflating: 18.txt       

Necessário executar somente se estiver no MAC

In [14]:
!cd "Modera v2"/ && rm -rf __MACOSX

Deletando arquivo .zip

In [15]:
!cd "Modera v2"/ && rm "Modera v2.zip"

Escolhendo a unidade que desejar

In [25]:
company = Company.objects.get(name="Modera")
company

<Company: 31ba4470-cdfd-48ff-a513-8d9859f4a4a9: Modera>

Carregando e contando os OccurrenceTypes já existentes no sistema (para utilização na parte opcional)

In [26]:
old_forms = OccurrenceType.objects.filter(company=company)
for a in old_forms:
    print(a.name)

Carregando os dados dos arquivos .txt gerados

In [27]:
new_forms = []

for index, file in enumerate(os.listdir('Modera v2')):
    if not '.txt' in file:
        continue
    file_name = 'Modera v2/{}'.format(file)
    print('------')
    print(file_name)
    with open(file_name, 'r+') as fo:
        form = json.load(fo)
        print(form['displayName'])
        new_forms.append(form)

------
Modera v2/34.txt
Drenagem de túneis
------
Modera v2/12.txt
Reposição de material em vias secundárias não pavimentadas
------
Modera v2/7.txt
Reparo/correção de panelas ou buracos - Pav. rígido
------
Modera v2/24.txt
Remoção de animais mortos e sepultamento
------
Modera v2/67.txt
Instalação de sinalização de balizadores
------
Modera v2/51.txt
Controle de processos erosivos e de assoreamento
------
Modera v2/57.txt
Posicionamento de sinalização vertical
------
Modera v2/81.txt
Limpeza ou pintura das superfícies expostas ao tráfego
------
Modera v2/65.txt
Posicionamento de sinalização de defensa metálica
------
Modera v2/1.txt
Reparo/correção de panela ou buraco - Pav. flexível/Semi rígido
------
Modera v2/74.txt
Retrorefletividade de tachas refletivas
------
Modera v2/26.txt
Recomposição de erosão em aterro e/ou corte.
------
Modera v2/5.txt
Selagem de trincas
------
Modera v2/70.txt
Retrorefletividade de sinalização de balizadores
------
Modera v2/66.txt
Retrorefletividade de

Testando form para conferir se dados foram carregados corretamente

In [28]:
new_forms[0]

{'id': 34,
 'name': 'Drenagem de túneis',
 'displayName': 'Drenagem de túneis',
 'fields': [{'displayName': 'Comprimento',
   'apiName': 'comprimento',
   'dataType': 'float',
   'unit': 'm',
   'id': 1},
  {'displayName': 'Largura',
   'apiName': 'largura',
   'dataType': 'float',
   'unit': 'm',
   'id': 2},
  {'displayName': 'Espessura',
   'apiName': 'espessura',
   'dataType': 'float',
   'unit': 'cm',
   'id': 3},
  {'displayName': 'Área',
   'apiName': 'area',
   'dataType': 'float',
   'autofill': {'if': [{'missing': ['formData.largura',
       'formData.comprimento']},
     0,
     {'*': [{'var': 'formData.largura'}, {'var': 'formData.comprimento'}]}]},
   'disableIf': True,
   'unit': 'm²',
   'id': 4},
  {'displayName': 'Volume',
   'apiName': 'volume',
   'dataType': 'float',
   'autofill': {'if': [{'missing': ['formData.largura',
       'formData.comprimento',
       'formData.espessura']},
     0,
     {'*': [0.01,
       {'var': 'formData.largura'},
       {'var': 'formD

Escolhendo nome do arquivo Excel para carregar as Classes

In [29]:
filename = "Modera v2/Modera v2"

Escolhendo a aba da planilha do Excel
Obs: atenção para escolher a aba correta da planilha

In [30]:
wb = load_workbook(filename + ".xlsx")
# sheetname = wb.sheetnames[8]
ws = wb['Classes']

Testando planilha carregada do Excel

In [31]:
print(ws['A1'].value)

Cor


Carregando as classes no script

In [32]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

Testando as configurações das Classes para conferir se dados foram carregados corretamente

In [33]:
values[0]

{'Cor': None,
 'Classe': 'Reparo/correção de panela ou buraco - Pav. flexível/Semi rígido',
 'id': 1,
 'Prazo': None,
 'Natureza': 'Pavimento Flexível e Semi rígido',
 'id_Natureza': 3}

Inserindo as informações de Natureza e Prazo no form

In [34]:
for a in values:
    if a['Classe'] not in [x['displayName'] for x in new_forms]:
        print(a['Classe'])

In [35]:
for a in new_forms:
    if a['displayName'] not in [x['Classe'] for x in values]:
        print(a['displayName'])

In [36]:
for form in new_forms:
    if form['displayName'] not in [a['Classe'] for a in values]:
        print(form['displayName'])
    else:
#         row = next(a for a in values if a['id'] == form['id'])
        row = next(a for a in values if a['Classe'] == form['displayName'])
#         print(form['displayName'], row['id_Natureza'])
        form['kind'] = row['id_Natureza']
#         print(form['kind'])

        if row['Prazo']:
            if 'horas' in row['Prazo']:
                form['deadline'] = timedelta(hours=int(row['Prazo'].split(' ')[0]))
            elif 'dias' or 'dia' in row['Prazo']:
                form['deadline'] = timedelta(days=int(row['Prazo'].split(' ')[0]))
            else:
                print('DEU RUIM')

Testando o form modificado

In [37]:
new_forms[0]

{'id': 34,
 'name': 'Drenagem de túneis',
 'displayName': 'Drenagem de túneis',
 'fields': [{'displayName': 'Comprimento',
   'apiName': 'comprimento',
   'dataType': 'float',
   'unit': 'm',
   'id': 1},
  {'displayName': 'Largura',
   'apiName': 'largura',
   'dataType': 'float',
   'unit': 'm',
   'id': 2},
  {'displayName': 'Espessura',
   'apiName': 'espessura',
   'dataType': 'float',
   'unit': 'cm',
   'id': 3},
  {'displayName': 'Área',
   'apiName': 'area',
   'dataType': 'float',
   'autofill': {'if': [{'missing': ['formData.largura',
       'formData.comprimento']},
     0,
     {'*': [{'var': 'formData.largura'}, {'var': 'formData.comprimento'}]}]},
   'disableIf': True,
   'unit': 'm²',
   'id': 4},
  {'displayName': 'Volume',
   'apiName': 'volume',
   'dataType': 'float',
   'autofill': {'if': [{'missing': ['formData.largura',
       'formData.comprimento',
       'formData.espessura']},
     0,
     {'*': [0.01,
       {'var': 'formData.largura'},
       {'var': 'formD

Criando novos OccurrenceTypes com as informações dos forms modificados

In [38]:
occurrence_types = []
counter=0

for form in new_forms:
    form_copy = deepcopy(form)
    if 'deadline' in form_copy.keys():
        del form_copy['deadline']
    if 'kind' in form_copy.keys():
        del form_copy['kind']
    
    occurrence_type = OccurrenceType(
        name=form['displayName'].strip(),
        occurrence_kind=form['kind'],
        form_fields=form_copy,
        created_by=User.objects.get(username='rlcs'),
        deadline= form['deadline'] if 'deadline' in form else None
    )
#     print(occurrence_type)
    occurrence_types.append(occurrence_type)
    counter+=1

print("\n")
print(counter)



88


Testando OccurrenceType criado

In [39]:
occurrence_types[0].form_fields

{'id': 34,
 'name': 'Drenagem de túneis',
 'displayName': 'Drenagem de túneis',
 'fields': [{'displayName': 'Comprimento',
   'apiName': 'comprimento',
   'dataType': 'float',
   'unit': 'm',
   'id': 1},
  {'displayName': 'Largura',
   'apiName': 'largura',
   'dataType': 'float',
   'unit': 'm',
   'id': 2},
  {'displayName': 'Espessura',
   'apiName': 'espessura',
   'dataType': 'float',
   'unit': 'cm',
   'id': 3},
  {'displayName': 'Área',
   'apiName': 'area',
   'dataType': 'float',
   'autofill': {'if': [{'missing': ['formData.largura',
       'formData.comprimento']},
     0,
     {'*': [{'var': 'formData.largura'}, {'var': 'formData.comprimento'}]}]},
   'disableIf': True,
   'unit': 'm²',
   'id': 4},
  {'displayName': 'Volume',
   'apiName': 'volume',
   'dataType': 'float',
   'autofill': {'if': [{'missing': ['formData.largura',
       'formData.comprimento',
       'formData.espessura']},
     0,
     {'*': [0.01,
       {'var': 'formData.largura'},
       {'var': 'formD

## Atenção! Este passo modifica informações do banco de dados

Salva os OccurrenceTypes criados no sistema, assim como cria e salva os OccurrenceTypeSpecs

In [40]:
for occurrence_type in tqdm(occurrence_types):
#     if occurrence_type.name.strip() not in [a.name.strip() for a in old_forms]:
    occurrence_type.save()
    spec = OccurrenceTypeSpecs(
        occurrence_type = occurrence_type,
        company = company,
        color = "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    )
    spec.save()

  0%|          | 0/88 [00:00<?, ?it/s]

# Começo da parte opcional

Escolhendo a aba de "De/Para" e carregando os dados no sistema
Obs: atenção para escolher a aba correta da planilha

In [39]:
# Opcional

wb = load_workbook(filename + ".xlsx")
sheetname = wb.sheetnames[2]
ws = wb[sheetname]

header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

/home/ubuntu/hidros-backend/venv/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Testando para ver se os dados foram carregados com sucesso

In [26]:
values[0]
# occurrence_types[0].__dict__
# old_forms[0].name
# type(old_forms)
# old_forms[0].__dict__

{'Natureza': 'Apontamento',
 'Classe Antiga': 'CAIAÇÃO COM FIXADOR DE CAL',
 'Classe Nova': 'CAIAÇÃO COM FIXADOR DE CAL',
 None: None,
 'Legenda': 'Não há correspondencia',
 'Normal': 'Sem correspondencia "para"'}

In [40]:
# Opcional

for value in values:
    if value['Classe Antiga']:
        old_form = old_forms.filter(name=value['Classe Antiga']).first()
#         print(value['Classe Antiga'])
        new_form = [occ for occ in occurrence_types if occ.name == value['Classe Nova']][0]
        new_form.previous_version = old_form
        print(new_form.name, '\t', new_form.previous_version.name)

CAIAÇÃO COM FIXADOR DE CAL 	 CAIAÇÃO COM FIXADOR DE CAL
FORMAS DE TÁBUAS DE PINHO PARA DISPOSITIVOS DE DRENAGEM 
UTILIZAÇÃO DE 3 VEZES - CONFECÇÃO, INSTALAÇÃO E RETIRADA 	 Colocação de gabaritos
CONCRETO CICLÓPICO FCK = 20 MPA - CONFECÇÃO EM BETONEIRA E LANÇAMENTO MANUAL -
AREIA, BRITA E PEDRA DE MÃO COMERCIAIS 	 CONCRETO CICLÓPICO FCK = 20 MPA
CONCRETO FCK = 20 MPA - CONFECÇÃO EM BETONEIRA E LANÇAMENTO MANUAL - AREIA E BRITA COMERCIAIS 
AREIA, BRITA E PEDRA DE MÃO COMERCIAIS 	 Concreto Comum
CORREÇÃO DE DEFEITOS COM MISTURA BETUMINOSA, INCLUSIVE PINTURA DE LIGAÇÃO 	 Correção de Defeitos
DEMOLIÇÃO MANUAL DE MEIO FIO DE CONCRETO 	 DEMOLIÇÃO MANUAL DE MEIO FIO DE CONCRETO
DESTOCAMENTO DE ÁRVORES COM DIÂMETRO DE 0,15 A 0,30 M 	 DESTOC. ÁRVORES D= 0,15 À 0,3 M
DISSIPADOR DE ENERGIA - DEB 03 - AREIA E PEDRA DE MÃO COMERCIAIS 	 DISSIPADOR DE ENERGIA - DEB 03 - AREIA E PEDRA DE MÃO COMERCIAIS
ESCAVAÇÃO MANUAL EM MATERIAL DE 1ª CATEGORIA 	 Escavação manual
ESCAVAÇÃO MECÂNICA DE VALA EM MATERIA

Testando forms atualizados com o "De/Para"

In [86]:
# new_form

<OccurrenceType: Inspeções de Pontes, Viadutos, Túneis e Passarelas - ['Arteris Intervias']>

Todos os forms antigos serão desativados?

In [41]:
for a in old_forms:
    a.active=False
#     a.save()

# Fim da parte opcional